In [31]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile

from bs4 import BeautifulSoup
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license

from le_utils.constants import content_kinds, file_formats, licenses


In [32]:
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108'

In [37]:

BG_IMG_RE = re.compile("background-image:url\((.*)\)")
SEND_FACEBOOK_RE = re.compile("sendFacebook\([^,]*,[^,]*,(.*)\)")


def download_module(module_url):
    print('Scapring module...')
    doc = get_parsed_html_from_url(module_url)
    destination = tempfile.mkdtemp()
    print('destination=', destination)
    
    section_lis = doc.find('ul', class_='oucontent-tree-current-section').find_all('li', recursive=False)
    
    contents_dict = {}
    
    for section_li in section_lis:
        print(section_li.find('a')['href'])
        subsection_lis = section_li.find('ul').find_all('li')
        
        for subsection_li in subsection_lis:
            print('   ', subsection_li.find('a')['href'])


    with open(os.path.join(destination, "index.html"), "w") as f:
        f.write(str(doc))

    # zip_path = create_predictable_zip(destination)

    source_id = parse_qs(urlparse(module_url).query)['id'][0]
    raw_title = doc.select_one("head title").text
    title = raw_title.replace('African Storybook -', '').strip()

    module_dict = dict(
        source_id=source_id,
        title=title,
        license=get_license(licenses.CC_BY_NC_SA, copyright_holder='TESSA'),
        description='fake descri',
        files=[], # files.HTMLZipFile(zip_path)],
    )
    return module_dict



def download_page(page_url):
    print('Scapring module page...')
    pass
    

In [38]:
download_module(test_module)

destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmppq0uj323
Scapring module...
http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.1
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.1
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.2
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.3
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.4
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.5
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.6
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.7
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=1.8
http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&section=2
    http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108&secti

{'description': 'fake descri',
 'files': [],
 'license': <ricecooker.classes.licenses.CC_BY_NC_SALicense at 0x115744a58>,
 'source_id': '81108',
 'title': 'OLCreate: TESSA_Eng Module 1: Personal development – how self-esteem impacts on learning'}

In [28]:

def passa():

    def download_assets(selector, attr, middleware=None):
        nodes = doc.select(selector)
        for i, node in enumerate(nodes):
            url = make_fully_qualified_url(node[attr])
            filename = "%s_%s" % (i, os.path.basename(url))
            node[attr] = filename
            download_file(url, destination, request_fn=make_request, filename=filename, middleware_callbacks=middleware)

    def js_middleware(content, url, **kwargs):
        # Polyfill window.localStorage as iframes can't access localStorage.
        return content.replace("window.localStorage",
                "({setItem: function(){}, removeItem: function(){}})")

    # Download all static assets.
    # TODO(davidhu): Also download fonts referenced in http://www.africanstorybook.org/css/app.css
    download_assets("img[src]", "src")  # Images
    download_assets("link[href]", "href")  # CSS
    download_assets("script[src]", "src", middleware=js_middleware) # JS

    # Download all background images, e.g. <div style="background-image:url()">
    # (africanstorybook.org uses these for the main picture found on each page
    # of the storybook.)
    bg_img_nodes = doc.select("div[style*=\"background-image:url(\"]")
    for i, node in enumerate(bg_img_nodes):
        style = node["style"]
        match = BG_IMG_RE.search(style)
        if not match:
            continue

        url = make_fully_qualified_url(match.group(1))
        filename = "%s_%s" % (i, os.path.basename(url))
        node["style"] = BG_IMG_RE.sub("background-image:url(%s)" % filename, style)
        download_file(url, destination, request_fn=make_request, filename=filename)

    # Hide the African Storybook header nav bar.
    header = doc.select_one("#headerBar")
    if header:
        header["style"] = "display: none;"
